# 数据挖掘-- 异常检测
MF1815083 高茜

MF1815098 陆峰

MG1815026 田甜
***
## 1.初步探索数据集信息&散点图识别异常值

* 一共有七个数据集，每个数据集都有两列数据：时间戳和value，提取时间戳特征中的月份和日期，看看数据来自怎样的时间段？
* 名字相似的数据集之间是否有一些关系可以辅助检测异常值？

In [ ]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 探索性数据分析
def EDA(x):
    data = pd.read_csv(x)
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    print(x + '数据集信息:')
    print(data.isnull().any())
    print(data['value'].describe())
    data['day'] = data['timestamp'].dt.day
    unique = data['timestamp'].dt.month.unique()
    print('月份：', unique) 
    if len(data['timestamp'].dt.month.unique()) <= 1:
        print('日期：', data['day'].unique(), '日期个数：', (len(data['day'].unique())))
    else:
        data['month'] = data['timestamp'].dt.month
        print((data.groupby('month')['month'].count()))
        for i in unique:
            tmp = data[data['month']==i]['day'].unique()
            print(str(i) + '月的日期：', tmp, '日期个数:', (len(tmp)))
    return data

def cut(name):
    '''
    散点图识别异常点
    '''
    if name == 'occupancy_6005.csv':
        outlier = data.loc[data['value'] > 20, 'timestamp']
    elif name == 'occupancy_t4013.csv':
        outlier = data.loc[data['value'] > 35, 'timestamp']
    elif name == 'speed_6005.csv':
        outlier = data.loc[data['value'] < 40, 'timestamp']
    elif name == 'speed_7578.csv':
        outlier = data.loc[data['value'] < 10, 'timestamp']
    elif name == 'speed_t4013.csv':
        outlier = data.loc[data['value'] < 25, 'timestamp']
    elif name == 'TravelTime_387.csv':
        outlier = data.loc[data['value'] > 4000, 'timestamp']
    else:
        outlier = data.loc[data['value'] > 5000, 'timestamp']
    return outlier
names = ['occupancy_6005.csv', 'occupancy_t4013.csv', 'speed_6005.csv', 'speed_7578.csv', 'speed_t4013.csv',
            'TravelTime_387.csv', 'TravelTime_451.csv']
result = pd.read_csv('anomalyInTrafficData.csv')
result.columns = ['TravelTime_387.csv', 'TravelTime_451.csv',
                                                       'occupancy_6005.csv', 'occupancy_t4013.csv',
                                                       'speed_6005.csv', 'speed_7578.csv', 'speed_t4013.csv']
def IF(data):
    '''
    孤立森林
    '''
    data.drop(['day','timestamp'], axis=1, inplace=True)
    clf = IsolationForest(n_estimators=100,  # 构建100棵树
                          contamination=0.0012)
    clf.fit(data)
    output = pd.DataFrame(clf.predict(data))
    return output

def evaluate(set1, set2, title):
    '''
    评估器：查准、查全、F1、准确率
    '''
    And = len(set1&set2)
    if And != 0:
        P = round(And / len(set1), 4)
        R = round(And / len(set2), 4)
        F1 = round(2 * P * R / (P + R), 4)
    else: P = R = F1 = 0
    print(title)
    print('P:',P,'R:',R,'F1:',F1)
    
for name in names:
    data = EDA(name)
    x = range(data.shape[0])
    plt.scatter(x, data['value'])
    plt.xticks([])
    plt.title(name + 'scatter')
    plt.show()
    plt.plot(data['timestamp'], data['value'])
    plt.xticks([])
    plt.title(name + 'line')
    plt.show()
    plt.boxplot(data['value'])
    plt.title(name + 'box')
    plt.show()
    # 散点图截断
    out = cut(name)
    # TP
    set1 = set(out)
    set2 = set(pd.to_datetime(result[name]))
    evaluate(set1, set2, '散点图截断评估')
    # 孤立森林
    data_time = data['timestamp']
    output = IF(data)
    outlier_index = output[(output[0]==-1)].index.tolist()
    # 孤立森林找出来的异常点所对应的时间
    set3 = set(data_time[outlier_index])
    evaluate(set3, set2, '孤立森林评估')

## 结论

| 数据集名称 | 样本量 | 均值 | 日期 | Precision | Recall | F1 | 
| --- | :---: | :---: | :--- | --- | --- | --- |
| occupancy_6005 | 2380 | 4.495 | 9月:1 2 3 4 8 9 10 11 12 13 14 15 16 17 |0.33 | 0.5 | 0.4 |
| occupancy_t4013 | 2500 | 7.243 | 9月:1 2 3 4 8 9 10 11 12 13 14 15 16 17| 0.5 | 0.67 | 0.57 |
| speed_6005 | 2500(23,2477) | 81.907 | 8月:31;  9月:1 2 3 4 8 9 10 11 12 13 14 15 16 17|  0.33 | 0.5 | 0.4 |
| speed_7578 | 1127 | 64.049 | 9月:8 9 10 11 12 13 14 15 16 17| 0.25 | 0.5 | 0.33 |
| speed_t4013 | 2495 | 62.934 | 9月:1 2 3 4 8 9 10 11 12 13 14 15 16 17| 0.2 | 0.67 | 0.31 |
| TravelTime_387 | 2500(490,1030,980) | 325.094 | 7月:10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31; 8月:31天都有; 9月:1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17| 0.2 | 0.25 | 0.22 |
| TravelTime_451 | 2162(89,1183,890) | 327.222 | 7月:28 29 30 31; 8月:31天都有;9月:1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17| 0 | 0 | 0|

* 数据集无缺失值
* 时间跨度是7，8，9月，数据集之间不能合并，因为例如occupancy两个数据集中相同时间戳的value值不同，且总体来看均值差异较大，说明对象不是同一批，可能是因为车型不同，速度量级不同
* 数据是有序的，测量的是某一段时间内离散的值
* 可以对各个数据集用相同的模型来检测异常点
* 用箱线图无法识别异常值，箱线图只能显示对于整体数据集来说极大或极小的值，对这几个数据集来说并不合适，可能是因为有时序信息
* 从折线图可以看出，名字相同的数据集之间变化趋势相近，各个数据集都显示了周期性波动的规律
* 从散点图可以看出，离群点的判定界限不明显，需要人工干预
* 根据散点图截断识别异常值，给出查准，查全和F1的值，可以看出，查准率在各个数据集上普遍不高于0.5，查全率都高于0.5，对于这个问题来说直观上看，查全率更可靠，因为我们更关注真正异常的值有没有被识别出来，而不是识别出来的值有多少是真正的异常值。比如说要识别出逃犯的车，那我们更关心所有的逃犯是否被识别出来。

## 2.Isolation Forest(孤立森林)识别异常值

* 散点图识别异常值的方式过于主观，并且没有用到密度信息，对于该数据集来说，一个值是否异常并非看它是否是离群点，而是要看在一个时间临域内这个点的表现是否异常，如果在这个时间临域内，这个点的value是异于其他的点，那它很可能就是异常值点
* 不能用基于聚类或者分类的方法，因为异常点实在是太少了
* 尝试使用孤立森林，因为随机森林的思想是当异常点只有极少量的时候，对数据进行随机划分，异常点会很快被孤立起来，用value来建树
* 孤立森林有两个超参数，n_estimator树的个数，实践表明取100会比较好，多余100棵树效果不会更好；contamination异常数据比例，比例的设置会很大地影响结果，取了几组contamination来看效果，发现取0.0012时最好
* 孤立森林由于采用类似CART树随机二分的方式，使得它既可对离散属性操作也可对连续属性操作
* 从第一部分可以看出，数据的周期性明显，为了能让孤立森林识别数据中的结构信息，考虑抽取时间相关特征，比如抽取日期特征，效果不好，猜测可能是因为周期性是体现在按小时划分上，因此抽取时间段特征，效果也不好
* 最终就是用vlaue单特征做孤立森林

|数据集名称|方法|Precision|Recall|F1|方法|Precision|Recall|F1|
|---|---|---|---|---|---|---|---|---|
|occupancy_6005|IF|0.33|0.5|0.4|scatter|0.33|0.5|0.4|
|occupancy_t4013|IF|0.67|0.67|0.67|scatter|0.5|0.67|0.57|
|speed_6005|IF|0.33|0.5|0.4|scatter|0.33|0.5|0.4|
|speed_7578|IF|0.5|0.25|0.33|scatter|0.25|0.5|0.33|
|speed_t4013|IF|0.67|0.67|0.67|scatter|0.2|0.67|0.31|
|TravelTime_387|IF|0|0|0|scatter|0.2|0.25|0.22|
|TravelTime_451|IF|0|0|0|scatter|0|0|0|

* 对比发现，孤立森林相比于散点图截断方法提高了Precision，但是，孤立森林没有识别出第六七个数据集的异常值，散点图截断没有识别出第七个数据集的异常值